In [ ]:
# Code repurposed for testing from 
#https://www.kaggle.com/kazmiekr/predicting-nhl-hall-of-famers/comments

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNBS
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import imblearn
from imblearn.ensemble import BalancedRandomForestClassifier


%matplotlib inline
import tensorflow as tf

In [ ]:
master_df = pd.read_csv('Jeff/Output/test_train_file.csv')
master_df.head()

In [ ]:
master_df.columns

In [ ]:
new_hof = ['Jarome Iginla', 'Marian Hossa','Martin St. Louis','Teemu Selanne','Nicklas Lidstrom','Chris Pronger','Mark Recchi','Mike Modano','Peter Forsberg']
new_hof

In [ ]:
#drop hof additions since 2011 for testing purposes
inverse_boolean_series = ~master_df.Player.isin(new_hof)
retired_players = master_df[inverse_boolean_series]

retired_players = retired_players[retired_players['GP']>=100]

#create df with only new hof additions for testing
boolean_series = master_df.Player.isin(new_hof)
new_hof_df = master_df[boolean_series]
# print(new_hof_df)


initial_eligible_players = pd.read_csv('Jeff/Output/predict_file.csv')

#isolate any 'active' hof in the eligible players dataset
active_hof = initial_eligible_players[initial_eligible_players['HoF'] == 1]
# print(active_hof)
#append any 'active' hof players to the test set
new_hof_df = new_hof_df.append(active_hof)

#update the eligible players df
eligible_players = initial_eligible_players[initial_eligible_players['HoF'] == 0]
# print(new_hof_df)

In [ ]:
cols_to_drop = ['Player','firstYear', 'lastYear','POS', 'HoF']

In [ ]:
# print(new_hof_df)

In [ ]:
#Training Set
train=retired_players.sample(frac=0.70, random_state=16)
# print(train.shape)

#Test Set
test=retired_players.drop(train.index)
# print(test.shape)

X_train = train.drop(cols_to_drop, axis=1)
Y_train = train['HoF']
X_test = test.drop(cols_to_drop, axis=1).copy()
Y_test = test['HoF']

# Dataset with unknown HoF predictions
X_player_test = eligible_players.drop(cols_to_drop, axis=1).copy()

# Create a test set of players that have made the HF since the last year of available data in this data set
# We'll just fill the Y with 1's
X_new_hof_test = new_hof_df.drop(cols_to_drop, axis=1).copy()
Y_new_hof_test = np.ones(X_new_hof_test.shape[0], dtype=np.int)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape


In [ ]:
classifiers = {}
# Run a classifier instance against the training set
def run_classifier(classifier):
    # Get the name of the classifier instance
    class_name = classifier.__class__.__name__
    print("Testing {0}".format(class_name))
    print("----------------------------------------\n")
    
    # Fit the classifier to the training set
    classifier.fit(X_train, Y_train)
    # Find out the training accuracy
    training_accuracy = round(classifier.score(X_train, Y_train) * 100, 2)
    print("Training Score: {0}".format(training_accuracy))
    
    # Use the trained data to predict the next HoF's
    Y_pred = classifier.predict(X_test)
    
    # How accurate was it against the known set of recent HoF's
    hof_accuracy = test_current_hf(classifier)
    
    # Cross validation scores
    f1_score = f1(Y_test, Y_pred)
    
    
    print("\n----- Predictions -----\n")
    # For all the classifiers except LinearSVC, we'll determine the probability in addition
    # to the prediction
    if class_name != 'LinearSVC':
        (positive_predictions, _) = show_prediction_with_probabilities(classifier, X_player_test)
    else:
        (positive_predictions, _) = show_prediction(classifier, X_player_test)
        
    # Store our results per classifier for later analysis    
    classifiers[class_name] = {
        'Training Accuracy': training_accuracy,
        'F1 Score': f1_score,
        'Current HoF Accuracy': hof_accuracy
    }
    return positive_predictions

# Given our set of known recent HoF additions, does the classifier think they should have made it?
def test_current_hf(classifier):
    accuracy = round(classifier.score(X_new_hof_test, Y_new_hof_test) * 100, 2)
    print("Accuracy with Current HF: {0}".format(accuracy))
    return accuracy

# Display from cross validation scores, F1, precision, recall
def f1(Y_test, Y_pred):
    macro = f1_score(Y_test.values, Y_pred, average='macro')
    print("F1 Score: {0}".format(formatPercent(macro)))
    precision = precision_score(Y_test.values, Y_pred, average='macro')
    print("Precision Score: {0}".format(formatPercent(precision)))
    recall = recall_score(Y_test.values, Y_pred, average='macro')
    print("Recall Score: {0}".format(formatPercent(recall)))
    return macro

def formatPercent(num):
    return round(num, 2)
    
def show_prediction_with_probabilities(classifier, X_player_test, display=True):
    prediction = classifier.predict(X_player_test)
    prob = classifier.predict_proba(X_player_test)
    pred_df = pd.DataFrame({
#         "playerID": eligible_players["playerID"],
        "Player": eligible_players["Player"],
        "isInHfPrediction": prediction,
        #"prob1": prob[:,0],
        "probability": prob[:,1]
    })
    positive_predictions = pred_df[pred_df['isInHfPrediction'] == 1]
    if display == True:
        print(positive_predictions.sort_values(by='probability', ascending=False))    
    return positive_predictions, pred_df

def show_prediction(classifier, X_player_test):
    prediction = classifier.predict(X_player_test)
    pred_df = pd.DataFrame({
#         "playerID": eligible_players["playerID"],
        "Player": eligible_players["Player"],
        "isInHfPrediction": prediction
    })
    positive_predictions = pred_df[pred_df['isInHfPrediction'] == 1]
    print(positive_predictions)
    return positive_predictions, pred_df

In [ ]:
nn = MLPClassifier(alpha=1)
nn_pred = run_classifier(nn)

In [ ]:
logreg = LogisticRegression()
logreg_pred = run_classifier(logreg)

In [ ]:
svc = SVC(probability=True)
svc_pred = run_classifier(svc)

In [ ]:
coeff_df = pd.DataFrame(retired_players.columns.drop(['Player','firstYear', 'lastYear','POS', 'HoF']))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn_pred = run_classifier(knn)

In [ ]:
gaussian = GaussianNB()
gaussian_pred = run_classifier(gaussian)

In [ ]:
linear_svc = LinearSVC()
linear_svc_pred = run_classifier(linear_svc)

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree_pred = run_classifier(decision_tree)

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest_pred = run_classifier(random_forest)

In [ ]:
model_performance = pd.DataFrame.from_dict(classifiers, orient='index')
model_performance.sort_values(by='F1 Score', ascending=False)

In [ ]:
gaussian_pred.sort_values(by='probability', ascending=False).head(20)